In [1]:
import requests
import pandas as pd
import json
import re

In [2]:
alerts_df = pd.read_csv("alerts.csv")

In [14]:
alerts_df.head(2)

,id,url,title,parkCode,description,category,relatedRoadEvents,lastIndexedDate
0,A3206816-F751-494A-A8C8-3A2027656038,NaN,Cochran Shoals - Interstate North Men's Restro...,chat,The men's restroom at Cochran Shoals - Interst...,Information,[],2025-09-30 00:00:00.0
1,89D019BD-F0A7-4DAA-8427-31C7ADCA4D7E,NaN,Trail Closures for Feral Hog Management 09/30,cong,"Weston Lake, Oakridge, River, & Kingsnake Trai...",Park Closure,[],2025-09-30 00:00:00.0


In [4]:
tdx_df = pd.read_csv("TDx_tracker.csv")

In [15]:
tdx_df.head(2)

,date,park,alert name,description,event type,road name,vehicle impact,event start date,alert_id,region,alpha,State
0,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T22:37:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
1,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T22:37:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR


In [19]:
tdx_df = tdx_df.loc[tdx_df['date']=="2025-09-19"]

In [10]:
len(alerts_df.loc[alerts_df['relatedRoadEvents']!="[]"]), len(alerts_df)

(25, 726)

In [13]:
print(round(len(alerts_df.loc[alerts_df['relatedRoadEvents']!="[]"])/len(alerts_df)*100,1), "% of alerts tagged to a TDx alert")

3.4 % of alerts tagged to a TDx alert


In [20]:
len(alerts_df), len(tdx_df)

(726, 112)

In [44]:
road_labels = ['road', 'drive', 'bypass', 'highway', 'parking lot', 'parkway', 'route', 'street', 'bridge', 'roads']
closed_labels = ['closed', 'closure', 'closures', 'delays', 'delays']

road_pattern = "|".join(map(re.escape, road_labels))
closed_pattern = "|".join(map(re.escape, closed_labels))

road_label_matches = alerts_df['description'].str.contains(road_pattern, case=False, na=False)
closure_label_matches = alerts_df['description'].str.contains(closed_pattern, case=False, na=False)

alerts_df["Road_Closure"] = (road_label_matches & closure_label_matches).map({True: 'yes', False: 'no'})

In [45]:
alerts_df["Road_Closure"].value_counts()

Road_Closure
no     599
yes    127
Name: count, dtype: int64

In [46]:
alerts_df_roads = alerts_df.loc[alerts_df['Road_Closure']=="yes"]

In [47]:
alerts_df_noroads = alerts_df.loc[alerts_df['Road_Closure']=="no"]

In [50]:
len(tdx_df['park'].unique()), len(alerts_df['parkCode'].unique()), len(alerts_df_roads['parkCode'].unique())

(12, 307, 96)

In [51]:
len(tdx_df), len(alerts_df_roads)

(112, 127)

In [48]:
counter = 0

for x, y, z in zip(alerts_df_roads['description'], alerts_df_roads['parkCode'], alerts_df_roads['relatedRoadEvents']):
    print(y)
    print(x)
    if z!= "[]":
        print(z)
        counter+=1
    print('----------')

print(counter)

cong
Weston Lake, Oakridge, River, & Kingsnake Trails will be closed on Tuesday, September 30, from 7:00 AM to 3:00 PM for feral hog management work. Cedar Creek between Bannister Bridge & South Cedar Canoe Landing will also be closed during this time.
----------
grsm
Park Headquarters Road and a portion of the Gatlinburg Trail are closed through September 2025 due to rehabilitation work on the Sugarlands water and wastewater systems. During this time, 0.6 miles of the Gatlinburg Trail remain open and accessible from the downtown Gatlinburg trailhead. There is no access to the trail from the Sugarlands Visitor Center. The full trail is expected to reopen on October 3.
[{'title': 'Park Headquarters Road closed', 'id': '83FCF166-F81F-4492-A1D1-3013CD2A09D3', 'type': 'roadevent', 'url': 'https://www.nps.gov/grsm/planyourvisit/conditions.htm'}]
----------
keaq
Kenilworth Aquatic Gardens main entrance and parking lot will remain closed until December 31, 2025, at 11:59 pm. Visitors may ente

In [49]:
counter = 0

for x, y, z, zz in zip(alerts_df_noroads['description'], alerts_df_noroads['parkCode'], alerts_df_noroads['relatedRoadEvents'], alerts_df_noroads['title']):
    if z!= "[]":
        print(y)
        print(zz)
        print(x)
        print(z)
        counter+=1
        print('----------')

print(counter)

grca
North Rim Status and Inner Canyon Closures —Tuesday, September 30, 2025
Follow the link for North Rim status and current closures. ADDITIONAL CLOSURES: South Kaibab Trail below the Tipoff, Phantom Ranch and North Kaibab Trail.
[{'title': 'North Rim Roads Are Closed by the Dragon Bravo Wildfire', 'id': '872E3239-CDB2-4E89-8F57-1C87EC3FC3FE', 'type': 'roadevent', 'url': 'https://www.nps.gov/grca/planyourvisit/conditions.htm'}]
----------
moja
Delays up to 1 hour are possible at Kelbaker Road Railroad Crossing near Kelso Depot
In the event there are train siding operations at the crossing near Kelso Depot, delays may be up to one hour. Please plan accordingly for this possibility with extra fuel and drinking water. Union Pacific Contact: 1 800 848-8715
[{'title': 'Kelso Railroad Crossing Delays 1 hour or more', 'id': '7F8876D0-161A-470B-8A06-854B512C8FE8', 'type': 'roadevent', 'url': 'https://www.nps.gov/moja/planyourvisit/conditions.htm'}]
----------
grsm
Old NC 284 is closed from C

In [55]:
tdx_df.head()

,date,park,alert name,description,event type,road name,vehicle impact,event start date,alert_id,region,alpha,State
0,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T22:37:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
1,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T22:37:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
2,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T22:37:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
3,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T07:00:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
4,2025-09-19,Crater Lake National Park,East Rim Drive from Reflection Point to Sun Notch,East Rim Drive from Reflection Point to Sun No...,work-zone,['East Rim Drive'],all-lanes-closed,2025-07-18T07:00:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR


In [56]:
for x, y, z, zz in zip(tdx_df['description'], tdx_df['park'], tdx_df['alert_id'], tdx_df['alert name']):
    print(y)
    print(zz)
    print(x)
    print(z)
    print('----------')


Crater Lake National Park
East Rim Drive from Reflection Point to Sun Notch
East Rim Drive from Reflection Point to Sun Notch is closed for major road construction.  T
02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
----------
Crater Lake National Park
East Rim Drive from Reflection Point to Sun Notch
East Rim Drive from Reflection Point to Sun Notch is closed for major road construction.  T
02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
----------
Crater Lake National Park
East Rim Drive from Reflection Point to Sun Notch
East Rim Drive from Reflection Point to Sun Notch is closed for major road construction.  T
02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
----------
Crater Lake National Park
East Rim Drive from Reflection Point to Sun Notch
East Rim Drive from Reflection Point to Sun Notch is closed for major road construction.  T
02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
----------
Crater Lake National Park
East Rim Drive from Reflection Point to Sun Notch
East Rim Drive from Reflection Point to Sun Notch is closed 